In [112]:
import pandas as pd
import re

In [113]:
caminho = 'E:\\Python\\Juntar Dados Cadastrais e Contatos\\'

DadosCadastrais = 'BR2 - Enriquecimento Cliente - DB - Cortex'
SociosReceita = 'Receita Federal'
Contatos = 'Output_Contatos_Empresas_Linkedin_Apollo'
Tecnologias = 'Informações de Empresas Domínios'

df = pd.read_excel(caminho + DadosCadastrais + '.xlsx', sheet_name='Dados Cadastrais - Cortex')
df_socios = pd.read_excel(caminho + SociosReceita + '.xlsx', sheet_name='Adicionais')
df_contatos = pd.read_excel(caminho + Contatos + '.xlsx', sheet_name='Contatos')
df_tecnologias = pd.read_excel(caminho + Tecnologias + '.xlsx', sheet_name='Adicionais')
df_matriz_cnpj_website = pd.read_excel(caminho + DadosCadastrais + '.xlsx', sheet_name='Matriz CNPJ Website')
df_DQ = pd.read_excel(caminho + DadosCadastrais + '.xlsx', sheet_name='DQ')

C:\Users\Jose Augusto\AppData\Local\Programs\Python\Python39\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [114]:
def Remover_Caracteres_Especiais(item):

    return re.sub('[^A-Za-z] ', '', item.replace(".", " "))

In [115]:
def Retornar_Ultimo_Nome(item):
    item = item.split(' ')
    len(item)

    if(len(item) != 1):
        return item[len(item) - 1]
    else:
        ""

In [116]:
def Retornar_Cargo(item):

    item = str(item)

    item = item.split('-')

    if(len(item) == 2):
        return item[1]
    elif(len(item) == 3):
        return item[1] + ' ' + item[2]
    else:
        return ""

In [117]:
df_socios.rename(columns={'Nome':'NOME'}, inplace=True)

In [118]:
df_socios['NOME_FORMATADO'] = df_socios['NOME'].str.replace(" LTDA", "").str.replace(" EIRELI", "").str.replace(" S/A", "").str.replace(" SA", "").str.replace("S.A.", "").str.replace(" INC", "")
df_socios['NOME_FORMATADO'] = df_socios['NOME_FORMATADO'].apply(Remover_Caracteres_Especiais)

C:\Users\JOSEAU~1\AppData\Local\Temp/ipykernel_5884/4209885520.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df_socios['NOME_FORMATADO'] = df_socios['NOME'].str.replace(" LTDA", "").str.replace(" EIRELI", "").str.replace(" S/A", "").str.replace(" SA", "").str.replace("S.A.", "").str.replace(" INC", "")


In [119]:
df_socios['NOME_FORMATADO'] = df_socios['NOME_FORMATADO'].str.strip()

In [120]:
df_socios['PRIMEIRO NOME'] = df_socios['NOME'].str.split(' ', expand=True).get(0)
df_socios['ULTIMO NOME'] = df_socios['NOME_FORMATADO'].apply(Retornar_Ultimo_Nome)

In [121]:
df_socios['Cargo'] = df_socios['Cargo'].apply(Retornar_Cargo)

In [122]:
df_socios['NOME'] = df_socios['NOME'].str.title()

In [123]:
df_socios = df_socios.merge(df, left_on= 'QUERY', right_on= 'CNPJ', how = 'left')

In [124]:
df_socios['EMAIL'] = ''

In [125]:
df_socios = df_socios.filter(items=['CNPJ', 'RAZÃO SOCIAL', 'NOME', 'PRIMEIRO NOME', 'ULTIMO NOME', 'CARGO', 'EMAIL'])

In [126]:
df_socios_buscar_email = df_socios
df_socios_buscar_email = df_socios_buscar_email.merge(df_matriz_cnpj_website, on='CNPJ', how = 'inner')

In [139]:
df_socios_buscar_email['Input_AnyLeads'] = df_socios_buscar_email['PRIMEIRO NOME'].str.lower() + ',' + df_socios_buscar_email['ULTIMO NOME'].str.lower() + ',' + df_socios_buscar_email['Website'].str.lower() + ','

In [127]:
df_socios_buscar_email = df_socios_buscar_email.filter(items=['CNPJ', 'NOME', 'PRIMEIRO NOME', 'ULTIMO NOME', 'Website', 'Input_AnyLeads', 'EMAIL'])

In [129]:
df_tecnologias = df_tecnologias.merge(df_matriz_cnpj_website, left_on='QUERY', right_on='Website', how = 'inner')

In [130]:
df_tecnologias= df_tecnologias.filter(items=['CNPJ', 'category', 'name'])

In [131]:
df_tecnologias = df_tecnologias.merge(df, on='CNPJ', how='inner')

In [132]:
df_tecnologias = df_tecnologias.filter(items=['CNPJ', 'RAZÃO SOCIAL', 'category', 'name'])

In [133]:
df_tecnologias.rename(columns={'category': 'CATEGORIA', 'name': 'NOME'}, inplace=True)

In [134]:
df_contatos = df_contatos.merge(df_matriz_cnpj_website, left_on='WEBSITE', right_on='Website', how='inner')

In [135]:
df_contatos = df_contatos.filter(items=['CNPJ', 'PRIMEIRO NOME', 'ULTIMO NOME', 'CARGO', 'EMAIL', 'LINKEDIN_CONTATO'])

In [136]:
df_contatos = df_contatos.merge(df, on='CNPJ', how='inner')

In [137]:
df_contatos = df_contatos.filter(items=['CNPJ', 'RAZÃO SOCIAL', 'PRIMEIRO NOME', 'ULTIMO NOME', 'CARGO', 'EMAIL', 'LINKEDIN_CONTATO'])

In [140]:
#Cria um excel writer usando xlsxwriter como engine;
# Enriquecimento
# Extração
# Estudo
arquivo = 'PSNEW-9999'
cliente = 'Teste'
writer = pd.ExcelWriter(caminho + arquivo + ' - Enriquecimento ' + cliente + ' - DB - Cortex' + '.xlsx', engine='xlsxwriter')

#Escreve os dataframes em diferentes sheets dentro do arquivo final;
df_DQ.to_excel(writer, sheet_name='DQ', index=False)
df.to_excel(writer, sheet_name='Dados Cadastrais - Cortex', index=False)
df_matriz_cnpj_website.to_excel(writer, sheet_name='Matriz CNPJ Website', index=False)
df_socios.to_excel(writer, sheet_name='Dados Sócios - Cortex', index=False)
df_socios_buscar_email.to_excel(writer, sheet_name='Buscar Email Sócios', index=False)
df_contatos.to_excel(writer, sheet_name='Contatos B2B - Cortex', index=False)
df_tecnologias.to_excel(writer, sheet_name='Tecnologias Cloud - Cortex', index=False)

#Fecha o arquivo e salva no diretório;
writer.save()

Deselvolvido por: José Augusto Prodossimo Ferreira